In [1]:
#Copyright 2020 Vraj Shah, Arun Kumar
#
#Licensed under the Apache License, Version 2.0 (the "License");
#you may not use this file except in compliance with the License.
#You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
#Unless required by applicable law or agreed to in writing, software
#distributed under the License is distributed on an "AS IS" BASIS,
#WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#See the License for the specific language governing permissions and
#limitations under the License.

import tensorflow_data_validation as tfdv
import pandas as pd
import pandas as pd
from pandas.api.types import is_numeric_dtype
from collections import Counter,defaultdict

from tensorflow_data_validation.utils.schema_util import get_categorical_features
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

2023-10-09 00:37:09.493674: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-09 00:37:09.497797: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-09 00:37:09.555432: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-09 00:37:09.555493: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-09 00:37:09.555546: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
testdf = pd.read_csv('../../Benchmark-Labeled-Data/data_test.csv')
test_metadata = pd.read_csv('../../Benchmark-Labeled-Data/Metadata/meta_data.csv')

print(len(testdf),len(test_metadata))
test_merged = pd.merge(testdf,test_metadata,on='Record_id')
print(len(test_merged))

print(test_merged)
y_true = test_merged.y_act.values.tolist()
print(y_true)
print(len(y_true))

1985 4233
1985
      Record_id                                  Attribute_name  \
0           285                     Net Income-Cont. Operations   
1           285                Earnings Before Interest and Tax   
2           285                                   After Tax ROE   
3           285                                        For Year   
4           285  Equity Earnings/Loss Unconsolidated Subsidiary   
...         ...                                             ...   
1980        328                                            item   
1981        343                                     load_weight   
1982       1970                                    SpecialNotes   
1983       2050                                         Website   
1984        222                                             NaN   

                  y_act  total_vals  num_nans     %_nans  num_of_dist_val  \
0               numeric        1781         0   0.000000             1693   
1               numeric   

In [3]:
cntExceptions = 0
y_tfdv = [0]*1985
prv_csv_name,csv_name = '',''
exception_indices = []

for index, row in test_merged.iterrows():       
    if index%100==0: print(index)
    col = row['Attribute_name']
    prv_csv_name = csv_name
    csv_name = '../../RawCSV/' + row['name']
    
    print(csv_name)
    print(col)
    
    if prv_csv_name != csv_name:  df = pd.read_csv(csv_name,encoding='latin1')

    try:
        df_col = df[[col]]
    except KeyError:
        print('Exception Index:')
        print(index)
        y_tfdv[index]=1
        exception_indices.append(row)
        cntExceptions += 1
        continue
        
    st_option = tfdv.StatsOptions(enable_semantic_domain_stats=True)
    
    try:
        stats = tfdv.generate_statistics_from_dataframe(df_col,stats_options=st_option)
    except:
        print('Exception Index:')        
        print(index)
        y_tfdv[index]=1
        exception_indices.append(index)        
        continue
        
    schema = tfdv.infer_schema(statistics=stats)
    
    categ_lst = get_categorical_features(schema)
    for x in categ_lst: 
        print('Categorical')
        y_tfdv[index]=1
        break
    
    xc = schema.feature
    for x in xc:
        cnt_NLD = str(x).count('natural_language_domain')
        cnt_TD = str(x).count('time_domain')

        if cnt_NLD:
            print('Sentence')
            y_tfdv[index] = 3
        if cnt_TD:
            print('Time')
            y_tfdv[index] = 2   
        
    print()

0
../../RawCSV/fundamentals.csv
Net Income-Cont. Operations

../../RawCSV/fundamentals.csv
Earnings Before Interest and Tax

../../RawCSV/fundamentals.csv
After Tax ROE

../../RawCSV/fundamentals.csv
For Year

../../RawCSV/fundamentals.csv
Equity Earnings/Loss Unconsolidated Subsidiary

../../RawCSV/fundamentals.csv
Other Investing Activities

../../RawCSV/fundamentals.csv
Deferred Liability Charges

../../RawCSV/fundamentals.csv
Short-Term Debt / Current Portion of Long-Term Debt

../../RawCSV/fundamentals.csv
Other Financing Activities

../../RawCSV/fundamentals.csv
Net Income Adjustments

../../RawCSV/fundamentals.csv
Fixed Assets

../../RawCSV/fundamentals.csv
Total Liabilities

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
expense_OtherInvestments

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
expense_DeathBenefits

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
expense_AdminExpenses

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
expense_RealEstate

../../RawCSV/PPD_PlanLev

/tmp/ipykernel_100808/4109185584.py:15: DtypeWarning: Columns (41,142) have mixed types. Specify dtype option on import or set low_memory=False.
  if prv_csv_name != csv_name:  df = pd.read_csv(csv_name,encoding='latin1')



../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
CoversStateEmployees
Categorical

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
GainLossPeriod

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
other

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
GeoGrowth_est

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
NormCostRate_EE

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
equities_international

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
NormCostRate_tot

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
ReqContRate_tot

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
PhaseInPercent

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
FixedIncome_tot

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
CostSharing

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
EmployerType

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
InvestmentReturn_5yr_est
Categorical

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
PlanClosed
Categorical

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
GainLossB

/tmp/ipykernel_100808/4109185584.py:15: DtypeWarning: Columns (40,141) have mixed types. Specify dtype option on import or set low_memory=False.
  if prv_csv_name != csv_name:  df = pd.read_csv(csv_name,encoding='latin1')



../../RawCSV/PPD_PlanLevel_2017_12_11.csv
expense_SurvivorBenefits

../../RawCSV/PPD_PlanLevel_2017_12_11.csv
expense_refunds

../../RawCSV/PPD_PlanLevel_2017_12_11.csv
expense_AdminExpenses

../../RawCSV/PPD_PlanLevel_2017_12_11.csv
expense_SecuritiesLending

../../RawCSV/PPD_PlanLevel_2017_12_11.csv
expense_alternatives

../../RawCSV/PPD_PlanLevel_2017_12_11.csv
CoversLocalEmployees
Categorical

../../RawCSV/PPD_PlanLevel_2017_12_11.csv
GainLossPeriod

../../RawCSV/PPD_PlanLevel_2017_12_11.csv
GainLossRecognition

../../RawCSV/PPD_PlanLevel_2017_12_11.csv
GrossReturns

../../RawCSV/PPD_PlanLevel_2017_12_11.csv
JudgesAttorneys

../../RawCSV/PPD_PlanLevel_2017_12_11.csv
PercentADEC

../../RawCSV/PPD_PlanLevel_2017_12_11.csv
TierID

../../RawCSV/PPD_PlanLevel_2017_12_11.csv
ReqContRate_ER

../../RawCSV/PPD_PlanLevel_2017_12_11.csv
FixedIncome_tot

../../RawCSV/PPD_PlanLevel_2017_12_11.csv
ActFundedRatio_GASB67

../../RawCSV/PPD_PlanLevel_2017_12_11.csv
ActFundedRatio_GASB

../../RawCSV

...

Categorical

../../RawCSV/Registrations_Master_Vic.csv
Region
Categorical

../../RawCSV/Registrations_Master_Vic.csv
State
Categorical

../../RawCSV/igfaFish-Database..csv
Albacore
Categorical

../../RawCSV/medium_articles.csv
Design better dataÂ tables
Categorical
Sentence

../../RawCSV/neighborhood-health-clinics-historical.csv
SERVICES
Categorical

../../RawCSV/crime_incident_data2008.csv
Neighborhood
Categorical

../../RawCSV/classifier_data_0.csv
issue_title
Categorical
Sentence

../../RawCSV/Platinum.csv
System
Categorical
Sentence

../../RawCSV/estimates_of_main_aggregates_gross_value_added_by_kind_of_economic_activity_at_constant_2005_prices_national_currency_data.csv
item
Categorical
Sentence

../../RawCSV/estimates_of_main_aggregates_gross_value_added_by_kind_of_economic_activity_at_current_prices_national_currency_data.csv
category
Categorical

../../RawCSV/language to country.csv
Language;Country
Categorical

../../RawCSV/AA_2015.csv
Headline
Categorical

../../RawCSV/AA_20

/tmp/ipykernel_100808/4109185584.py:15: DtypeWarning: Columns (5,7,8,13) have mixed types. Specify dtype option on import or set low_memory=False.
  if prv_csv_name != csv_name:  df = pd.read_csv(csv_name,encoding='latin1')


Categorical

../../RawCSV/deep_sea_corals.csv
EventID
Categorical

../../RawCSV/deep_sea_corals.csv
SampleID
Categorical

../../RawCSV/ted_main.csv
title
Categorical
Sentence

../../RawCSV/ted_main.csv
url
Categorical

../../RawCSV/ted_main.csv
event
Categorical

../../RawCSV/url_features.csv
URL
Categorical

../../RawCSV/dream_market_cocaine_listings.csv
vendor_link
Categorical

../../RawCSV/avengers.csv
URL
Categorical

../../RawCSV/shark_tank.csv
website
Categorical

../../RawCSV/Fortune 500 2017 - Fortune 500.csv
Website
Categorical

../../RawCSV/EFF-MuckRock 2016-2017 ALPR DATA.csv
R6. 2017 Detection Report
Categorical

../../RawCSV/EFF-MuckRock 2016-2017 ALPR DATA.csv
R5. 2016 Hit Report
Categorical

../../RawCSV/EFF-MuckRock 2016-2017 ALPR DATA.csv
R7. 2017 Hit Report
Categorical

../../RawCSV/EFF-MuckRock 2016-2017 ALPR DATA.csv
R1. Data Sharing Link
Categorical

../../RawCSV/EFF-MuckRock 2016-2017 ALPR DATA.csv
R3. 2017 Hit Ratio Report
Categorical

../../RawCSV/kansas.csv
url

/tmp/ipykernel_100808/4109185584.py:15: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  if prv_csv_name != csv_name:  df = pd.read_csv(csv_name,encoding='latin1')


Categorical
Sentence

../../RawCSV/official_country_data_table_3_2_individual_consumption_expenditure_of_households_npishs_and_general_government_at_current_prices_data.csv
sub_group
Categorical

../../RawCSV/official_country_data_table_4_1_total_economy_s_1_data.csv
item
Categorical
Sentence

../../RawCSV/austin_waste_and_diversion.csv
load_weight

../../RawCSV/full_data.csv
SpecialNotes
Categorical
Sentence

../../RawCSV/times-square-food-beverage-locations.csv
Website
Categorical

../../RawCSV/WDICountry-Series.csv
nan
Exception Index:
1984


In [4]:
dict_label_true = {
    'numeric': 0,
    'categorical': 1,
    'datetime': 2,
    'sentence': 3,
    'url': 4,
    'embedded-number': 5,
    'list': 6,
    'not-generalizable': 7,
    'context-specific': 8
}

y_true = [dict_label_true[str(i)] for i in y_true]

In [5]:
print(accuracy_score(y_true, y_tfdv))
print(confusion_matrix(y_true, y_tfdv))

0.5622166246851386
[[706   1   0   0   0   0   0   0   0]
 [137 294   0  26   0   0   0   0   0]
 [  3  65  69   4   0   0   0   0   0]
 [  0  45   0  47   0   0   0   0   0]
 [  0  32   0   0   0   0   0   0   0]
 [  0  97   0   2   0   0   0   0   0]
 [  1  56   0   0   0   0   0   0   0]
 [126  81   1   7   0   0   0   0   0]
 [133  40   1  11   0   0   0   0   0]]
